# Mystic Rose

Here's how to create a mystic rose using python and turtle

In [1]:
from enum import Enum
class Render(Enum):
    SVG = 1
    CANVAS = 2
    RAW = 3

In [2]:
# Configuration
N_POINTS = 7
RADIUS = 200
RENDER_TYPE = Render.SVG

In [3]:
import math
import sys

match RENDER_TYPE:
    case Render.SVG:
        from svg_turtle import SvgTurtle
        from IPython.core.display import HTML
    case Render.CANVAS:
        import ipyturtle3 as turtle 
    case Render.RAW:
        import turtle 

Rainbow Generator from [/do/blog/2023_02_11_mysticrose/rainbow/](rainbow.ipynb)

In [4]:
%run ./rainbow.ipynb

In [5]:
# Mystic Rose Drawing Functions
pi = math.pi

def distance(i, j, n_points):
    return min( abs(i-j), n_points - abs(i-j))

def PointsInCircle(r, n=10, start=0):
    for i in range(start, n):
        yield {
            "i": i,
            "coords": (math.cos(2*pi/n*i)*r, 
                       math.sin(2*pi/n*i)*r) 
        }
        
def decimalColour(t):
    return tuple(ti/255 for ti in t)
        
def uniqueLines(radius, n_points):
    lines = []
    for pointA in PointsInCircle(radius, n_points):
        for pointB in PointsInCircle(radius, 
                                     n_points, 
                                     pointA["i"]+1):
            lines.append({
                "dist": distance(pointA["i"], 
                                 pointB["i"], 
                                 n_points),
                "from": pointA,
                "to": pointB
            })
    return lines

def drawMysticRose(radius, n_points, turtle):
    
    lines = sorted(uniqueLines(radius, n_points), 
                   key=lambda d: - d['dist'])

    max_dist = int(lines[0]['dist'])
    colours = list(map(decimalColour, 
                   generateRainbowColoursIter(max_dist)))
    
    for line in lines:
        if int(line["dist"]) <= len(colours):
            turtle.pencolor(colours[line["dist"]-1])
        else:
            turtle.pencolor("black")
            
        turtle.penup()
        #display(line["from"]["coords"])
        turtle.goto(line["from"]["coords"])
        turtle.pendown()
        turtle.goto(line["to"]["coords"])
        turtle.penup()
    return f'{n_points} points, {len(lines)} lines' 
    #same as int(n_points * (n_points-1) / 2)

In [6]:
class TutleKeeper:
    def __init__(self, renderType=Render.SVG):
        self._turtle = None
        self.canvas = None
        self.renderType = renderType
        self._screen = None
        match self.renderType:
            case Render.SVG:
                None
            case Render.CANVAS:
                self.canvas = turtle.Canvas(width=RADIUS*2,
                                            height=RADIUS*2)                
                #display(self.canvas )
                self._screen = turtle.TurtleScreen(self.canvas )
                self._screen.delay(1)
            case Render.RAW:
                wn = turtle.Screen()
                wn.tracer(False)

    def displayable(self):
        match self.renderType:
            case Render.SVG:
                return HTML(self._turtle.to_svg())
            case Render.CANVAS:
                return self.canvas
            
    def preDisplay(self):
        match self.renderType:
            case Render.CANVAS:
                display(self.displayable())        
        return self
    
    def postDisplay(self):
        match self.renderType:
            case Render.SVG:
                display(self.displayable())
        return self
        
    def draw(self, fn, *args):
        self.preDisplay()
        result = fn(*args, self.turtle())
        self.postDisplay()
        display(result)
        return self
    
    def end(self):
        return None

    def turtle(self):
        if self._turtle == None:
            match self.renderType:
                case Render.SVG:
                    self._turtle = SvgTurtle(width=RADIUS*2,
                                             height=RADIUS*2)
                case Render.CANVAS:
                    self._turtle = turtle.Turtle(self._screen)
                case Render.RAW:
                    self._turtle = turtle #???

            self._turtle.speed(2)
        return self._turtle

In [7]:
for i in range(6,30):
    TutleKeeper(RENDER_TYPE).draw(drawMysticRose, RADIUS, i)

'6 points, 15 lines'

'7 points, 21 lines'

'8 points, 28 lines'

'9 points, 36 lines'

'10 points, 45 lines'

...

'25 points, 300 lines'

'26 points, 325 lines'

'27 points, 351 lines'

'28 points, 378 lines'

'29 points, 406 lines'

In [8]:
# Avoid screen closing
if not is_notebook():    
    turtle.getscreen()._root.mainloop()  # <-- run the Tkinter main loop

NameError: name 'is_notebook' is not defined

NOTE
https://github.com/altair-viz/altair/issues/329#issuecomment-473524751